# Data Cleaning

This notebook produces the clustering dataset from the original MIMIC-IV files and some derived MIMIC-IV files (see the README). If these files are in the correct positions then all that is required is running this code. It will output a cleaned csv file in the data directory. The whole notebook takes 5-10 minutes to run.

# Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'
import os
import time
import gzip
import dask.dataframe as dd
import statsmodels.formula.api as smf
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Read the data in

**NOTE:** The datasets are large. In this code they are read straight into memory, however, alternate code may be necessary for machines with lower RAM. If it is not possible to run this code directly, we recommend using GoogleBigQuery to produce smaller versions of the chartevents and labevents files. For example, you can query it to only return the instances with the relevant itemids (see how it is read in in this code for more details).

First, we expand all of the .gz files. Then, we read in the icu files and the second the hosp files.

In [ ]:
# Define function to expand .gz files
def decompress_gz_files(directory, chunk_size=1024*1024):
    gz_files = [f for f in os.listdir(directory) if f.endswith('.gz')]
    print(f'Found {len(gz_files)} .gz files in the directory.')
    if len(gz_files)>0:
        print("Starting to decompress .gz files...\n")

    # Loop over all .gz files
    for i, filename in enumerate(gz_files, start=1):
        original_filepath = os.path.join(directory, filename) # Create a path to the .gz file
        new_filepath = os.path.splitext(original_filepath)[0]  # Create a path for the decompressed file (removes .gz from filename)

        with gzip.open(original_filepath, 'rb') as f_in:
            with open(new_filepath, 'wb') as f_out:
                # Read and write the first chunk and measure the time taken
                start_time = time.time()
                chunk = f_in.read(chunk_size)
                f_out.write(chunk)
                elapsed_time = time.time() - start_time

                # Estimate total time for this file
                total_size = os.path.getsize(original_filepath)
                estimated_time = total_size / len(chunk) * elapsed_time

                print(f'Decompressing {filename} ({i} out of {len(gz_files)}), '
                      f'estimated time: {estimated_time:.2f} seconds')

                # Read and write the rest of the file in chunks
                while chunk := f_in.read(chunk_size):
                    f_out.write(chunk)

        # Remove the .gz file
        os.remove(original_filepath)


# Decompress the .gz files
decompress_gz_files('data/icu')
decompress_gz_files('data/hosp')

In [ ]:
# Initialize an empty dictionary to store icu data
icu = {}

# Get a list of all CSV files in the icu folder and just the selected ones to import
icu_csv_files = [file for file in os.listdir("data/icu") if file.endswith(".csv")]
icu_files_to_import = ['d_items.csv', 'procedureevents.csv', 'inputevents.csv', 'ingredientevents.csv', 'icustays.csv']

# Create dictionary of which columns to read in. Adjust columns for certain large files
d_usecols = {file:None for file in icu_files_to_import}
d_usecols['ingredientevents.csv'] = ['hadm_id', 'stay_id', 'starttime', 'endtime', 'itemid']
d_usecols['inputevents.csv'] = ['hadm_id', 'stay_id', 'starttime', 'endtime', 'itemid', 'ordercategorydescription', 'originalrate']

# Create dictionary to parse dates 
d_parse_dates = {file:None for file in icu_files_to_import}
d_parse_dates['ingredientevents.csv'] = ['starttime', 'endtime']
d_parse_dates['inputevents.csv'] = ['starttime', 'endtime']

# Read each CSV file and store it in a data frame. Note that this ignore chartevents
for file_name in icu_files_to_import:

    # Identify the path to the file
    file_path = os.path.join("data/icu", file_name)

    # Remove the ".csv" extension from the file name
    df_name = file_name[:-4]

    # Read the CSV file into a data frame
    icu[df_name] = pd.read_csv(file_path, usecols = d_usecols[file_name], parse_dates=d_parse_dates[file_name])

In [ ]:
# Loading chartevents to memory. Note that chartevents is a large file so we use dask to read it in chunks and filter it. This might take a while to run depending on your machine.
dask_dataframe = dd.read_csv("data/icu/chartevents.csv", dtype={'caregiver_id': 'float64', 'value': 'object', 'valuenum': 'float64', 'warning': 'float64'}, parse_dates=['charttime'])

# Define the list of itemids that we need for later. If using GoogleBigQuery to create a smaller dataset, use the following itemids.
LAPS_Vars = [
    225624, 225624, 227000, 227001, 228152, 229669, 220227, 226767, 226860, 226861, 226862, 226863, 226865, 227035, 228232, 220179,
    225309, 226850, 223761, 223762, 220045, 220210, 224688, 224690, 224745, 225475, 227032, 227047, 229425, 229563, 220050, 220052,
    220277]

# Filter to the itemids in the list and compute
icu['chartevents'] = dask_dataframe[dask_dataframe['itemid'].isin(LAPS_Vars)].compute()

In [ ]:
# Initialize an empty dictionary to store hosp frames
hosp = {}

# Get a list of all CSV files in the hosp folder and just the selected ones to import
hosp_csv_files = [file for file in os.listdir("data/hosp") if file.endswith(".csv")]
hosp_files_to_import = ['admissions.csv', 'd_labitems.csv', 'omr.csv', 'diagnoses_icd.csv', 'services.csv', 'patients.csv']

# Read each CSV file and store it in a data frame
for file_name in hosp_files_to_import:

# Identify the path to the file 
    file_path = os.path.join("data/hosp", file_name)

    # Remove the ".csv" extension from the file name
    df_name = file_name[:-4]
    
    # Read the CSV file into a data frame
    hosp[df_name] = pd.read_csv(file_path)

In [ ]:
# Loading labevents to memory. Note that labevents is a massive file so we use dask to read it in chunks and filter it. This takes a while to run.
dask_dataframe = dd.read_csv("data/hosp/labevents.csv", parse_dates=['charttime'])

# Define the list of itemids that we need for later
itemid_list = [
    50803, 50882, 52039, 50868, 52500, 50824, 50983, 52455, 52623, 50912, 51081, 51977, 52024, 52546,
    50820, 52041, 50831, 51094, 51491, 52730, 50813, 52442, 53154, 50824, 50983, 52455, 52623, 50885,
    50912, 51081, 51977, 52024, 52546, 50862, 51775, 51776, 51807, 51836, 51910, 51926, 52022, 53085,
    53116, 53138, 50809, 50931, 51478, 51981, 52027, 52569, 51221, 51480, 51638, 51639, 52028, 51300,
    51301, 51755, 51756, 53134, 50818, 52040, 50821, 52042, 51003, 50825]

# Filter to the itemids in the list and compute
hosp['labevents'] = dask_dataframe[dask_dataframe['itemid'].isin(itemid_list)].compute()

# Feature recreation

This section contains the code to recreate each feature in Vranas et al. 2017.

### 1. Age
This code produces the same output as the SQL implementation [here](https://github.com/MIT-LCP/mimic-iv/blob/master/concepts/demographics/age.sql).

In [ ]:
# Convert 'admittime' in admissions dataframe to datetime
hosp['admissions']['admittime'] = pd.to_datetime(hosp['admissions']['admittime'])

# Merge the dataframes on 'subject_id'
merged_df = pd.merge(hosp['admissions'], hosp['patients'], on='subject_id')

# Calculate the age at the time of admission
merged_df['age'] = merged_df.apply(
    lambda row: (row['admittime'].year - row['anchor_year']) + row['anchor_age'], 
    axis=1
)

# Select the required columns
derived_age = merged_df[['hadm_id', 'age']]

## 2. Comorbidity
Using the CCI instead of COPS II (used in Vranas et al. 2017). We use the CCI implementation from the MIMIC-IV Code Repository: [here](https://github.com/MIT-LCP/mimic-code/blob/main/mimic-iv/concepts/comorbidity/charlson.sql). 

In [ ]:
# Read in the CCI data
CCI = pd.read_csv("data/queried_data/CCI.csv")[['hadm_id', 'charlson_comorbidity_index']]

## 3. Emergency department admission
This is one of the more complex features in this work. See the Appendix in the paper for details about why we chose this measure.

In [ ]:
# Convert ED admission and outtime to datetime. Make ED_Admission, a dummy which is 1 if they went into the ED
hosp['admissions']['edregtime'] = pd.to_datetime(hosp['admissions']['edregtime'])
hosp['admissions']['edouttime'] = pd.to_datetime(hosp['admissions']['edouttime'])
hosp['admissions']['ED_LOS'] = hosp['admissions']['edouttime'] - hosp['admissions']['edregtime']
hosp['admissions']['ED_LOS'] = hosp['admissions']['ED_LOS'].dt.total_seconds() / (24 * 60 * 60)
hosp['admissions']['ED_Admission'] = hosp['admissions']['ED_LOS'].notna().astype(int)

# Include the length between admission and edregtime for good measure 
hosp['admissions']['admittime'] = pd.to_datetime(hosp['admissions']['admittime'])
hosp['admissions']['ED_gap'] = hosp['admissions']['edregtime'] - hosp['admissions']['admittime']
hosp['admissions']['ED_gap'] = hosp['admissions']['ED_gap'].dt.total_seconds() / (24 * 60 * 60)

# Export the created features
admission_measures = hosp['admissions'][['hadm_id', 'ED_Admission']].copy()

## 4. Need for surgery

In [ ]:
# Codes for surgery
surgery_codes = ['CSURG', 'NSURG', 'ORTHO', 'SURG', 'TSURG', 'VSURG', 'PSURG', 'TRAUM', 'OBS']

# Create a dummy
hosp['services']['Surgery_Admission'] = hosp['services']['curr_service'].apply(lambda x: x in surgery_codes).astype(int)

# Export out
surgery_out = hosp['services'][['hadm_id', 'Surgery_Admission']].copy()

# Merge duplicate rows if the hospitalisation has at least 1 surgery
surgery_out = surgery_out.groupby('hadm_id')['Surgery_Admission'].any().astype(int)

In [ ]:
# Obstetrical Patients
Obstetrical_hadm_ids = hosp['services'][hosp['services'].curr_service == 'OBS'].copy()

# List of hospitalisations with OBS
Obstetrical_hadm_ids = Obstetrical_hadm_ids.hadm_id.unique()

## 5. Code status 
Code people as the most medically severe code that they have. e.g. If they are DNR at any point then code them as DNR only. Don't distinguish between partial code / DNR.

In [ ]:
# Select rows where 'itemid' is in code_status_ids.itemid. There are: 223758, 228687
code_status_events = pd.read_csv("data/queried_data/codestatus.csv")

# Our grouping of full code, partial code and DNR
full_code = ["Full code"]
DNR = ["DNAR (Do Not Attempt Resuscitation)  [DNR]", "DNR (do not resuscitate)", "DNR / DNI", "Comfort measures only", "DNAR (Do Not Attempt Resuscitation) [DNR] / DNI", "DNI (do not intubate)"]

# Add new columns. NOTE: need value here not valuenum (null)
code_status_events["Full_code"] = code_status_events.value.apply(lambda x: x in full_code).astype(int)
code_status_events["DNR"] = code_status_events.value.apply(lambda x: x in DNR).astype(int)

# Reduce columns and perform the aggregation
code_details = code_status_events[['stay_id', 'Full_code', 'DNR']].groupby('stay_id').any().astype(int)

# Remove full_code from those with DNRs.
code_details.loc[code_details['DNR'] ==1, 'Full_code'] = 0

## 6. Severity of illness at hospital sdmission (LAPS II)
See later [calculation of LAPS II requires features we create later on]

## 7. Predicted hospital mortality at hospital admission
See later [calculation of LAPS II requires features we create later on]

## 8. Severity of illness on ICU admission

Using SAPS II instead of SAPS III. The code to create the SAPS II data can be found [here](https://github.com/MIT-LCP/mimic-code/blob/main/mimic-iv/concepts/score/sapsii.sql). You can also download the derived data directly from GoogleBigQuery.

In [ ]:
# Import SAPS II data
sapsii = pd.read_csv("data/queried_data/sapsii.csv").sort_values('stay_id')
sapsii = sapsii[['stay_id', 'sapsii']]

## 9. Days receiving benzodiazepines
NOTE: can be slow to run

In [ ]:
# A list of all benzodiazepines used in the MIMIC-IV, ICU data
benzos = ['Diazepam (Valium)', 'Lorazepam (Ativan)', 'Midazolam (Versed)']

# Associated itemids
benzos_itemids = icu['d_items'][icu['d_items'].label.isin(benzos)].itemid

# Consider just the treatments table.
benzos_table = icu['inputevents'][icu['inputevents'].itemid.isin(benzos_itemids)].copy()

# Generate 'date' columns by extracting the date part from starttime and endtime
benzos_table['startdate'] = benzos_table['starttime'].dt.date
benzos_table['enddate'] = benzos_table['endtime'].dt.date

# Generate 'date' range column for each row
benzos_table['date'] = benzos_table.apply(lambda row: pd.date_range(start=row['startdate'], end=row['enddate']).date.tolist(), axis=1)

# Drop unnecessary columns to save memory. This is quite a neat idea
benzos_table = benzos_table.drop(['starttime', 'endtime', 'startdate', 'enddate'], axis=1)

# Flatten the 'date' column, remove duplicates within each 'stay_id' group, and reset index
df_grouped = (benzos_table.explode('date')
                        .groupby('stay_id')
                        .date
                        .apply(lambda x: x.drop_duplicates().tolist())
                        .reset_index())

# Calculate the number of unique dates where benzos were given
benzos_info = df_grouped.copy()
benzos_info['benzos_days'] = benzos_info['date'].str.len()
benzos_info = benzos_info[['stay_id', 'benzos_days']]

## 10. Days receiving other sedatives / non-benzodiazepines
NOTE: can be slow to run

In [ ]:
# A list of all non-benzodiazepines and other sedatives used (not including opioids). Removed all missing ones! 
other_seds = ['Propofol', 'Dexmedetomidine (Precedex)', 'Pentobarbital',
'Ketamine', 'Haloperidol (Haldol)']

# Associated itemids
seds_itemids = icu['d_items'][icu['d_items'].label.isin(other_seds)].itemid

# Consider the table with just seds shown
seds_table1 = icu['procedureevents'][icu['procedureevents'].itemid.isin(seds_itemids)].copy() # Current nothing 
seds_table2 = icu['inputevents'][icu['inputevents'].itemid.isin(seds_itemids)].copy()
seds_table3 = icu['ingredientevents'][icu['ingredientevents'].itemid.isin(seds_itemids)].copy() # Current nothing 

# Merge them for good practice.
seds_table = pd.concat([seds_table1, seds_table2, seds_table3], axis = 0)

# Generate 'date' columns by extracting the date part from starttime and endtime
seds_table['startdate'] = seds_table['starttime'].dt.date
seds_table['enddate'] = seds_table['endtime'].dt.date

# Generate 'date' range column for each row. This is a very slow line
seds_table['date'] = seds_table.apply(lambda row: pd.date_range(start=row['startdate'], end=row['enddate']).date.tolist(), axis=1)

# Drop unnecessary columns to save memory
seds_table = seds_table.drop(['starttime', 'endtime', 'startdate', 'enddate'], axis=1)

# Flatten the 'date' column, remove duplicates within each 'stay_id' group, and reset index
df_grouped_seds = (seds_table.explode('date')
                                .groupby('stay_id')
                                .date
                                .apply(lambda x: x.drop_duplicates().tolist())
                                .reset_index())

# Calculate the number of unique dates where seds were given
seds_info = df_grouped_seds.copy()
seds_info['seds_days'] = seds_info['date'].str.len()
seds_info = seds_info[['stay_id', 'seds_days']]

## 11. Days receiving opiates
NOTE: can be slow to run

In [ ]:
# A list of all opiates used
opiates = ['Fentanyl (Concentrate)', 'Morphine Sulfate', 'Meperidine (Demerol)', 'Fentanyl (Push)', 'Hydromorphone (Dilaudid)',
'Methadone Hydrochloride', 'Fentanyl']

# Associated itemids
opiates_itemids = icu['d_items'][icu['d_items'].label.isin(opiates)].itemid

# Consider just the inputevents table. All items are in the inputevents
opiates_table = icu['inputevents'][icu['inputevents'].itemid.isin(opiates_itemids)].copy()

# Generate 'date' columns by extracting the date part from starttime and endtime
opiates_table['startdate'] = opiates_table['starttime'].dt.date
opiates_table['enddate'] = opiates_table['endtime'].dt.date

# Generate 'date' range column for each row
opiates_table['date'] = opiates_table.apply(lambda row: pd.date_range(start=row['startdate'], end=row['enddate']).date.tolist(), axis=1)

# Drop unnecessary columns to save memory
opiates_table = opiates_table.drop(['starttime', 'endtime', 'startdate', 'enddate'], axis=1)

# Flatten the 'date' column, remove duplicates within each 'stay_id' group, and reset index
df_grouped_opiates = (opiates_table.explode('date')
                                .groupby('stay_id')
                                .date
                                .apply(lambda x: x.drop_duplicates().tolist())
                                .reset_index())

# Calculate the number of unique dates where opiates were given
opiates_info = df_grouped_opiates.copy()
opiates_info['opiate_days'] = opiates_info['date'].str.len()
opiates_info = opiates_info[['stay_id', 'opiate_days']]

## 12. Hospital length of stay

In [ ]:
# Hospital LOS
hosp['admissions']['admittime'] = pd.to_datetime(hosp['admissions']['admittime'])
hosp['admissions']['dischtime'] = pd.to_datetime(hosp['admissions']['dischtime'])
hosp['admissions']['LOS'] = hosp['admissions']['dischtime'] - hosp['admissions']['admittime']
hosp['admissions']['hosp_LOS'] = hosp['admissions']['LOS'].dt.total_seconds() / (24 * 60 * 60)
hosp_los = hosp['admissions'][['hadm_id', 'subject_id', 'hosp_LOS']]

## 13. Hospital mortality

In [ ]:
vital_status_discharge = hosp['admissions'] [['hadm_id', 'subject_id', 'hospital_expire_flag']]
vital_status_discharge = vital_status_discharge.rename(columns = {'hospital_expire_flag' : 'vital_status_discharge'})

## 14. Discharge location

In [ ]:
# Add a discharged home column to the original DataFrame
hosp['admissions']['discharged_home'] = hosp['admissions']['discharge_location'].apply(lambda x: (x=="HOME") | (x == "HOME HEALTH CARE") | (x=="AGAINST ADVICE") | (x=="ASSISTED LIVING")).astype(int)

# Add a discharged to hospice column to the original DataFrame
hosp['admissions']['discharged_hospice'] = hosp['admissions']['discharge_location'].apply(lambda x: x=="HOSPICE").astype(int)

# Add a discharged to skilled nursing facility (SNF) column to the original DataFrame
options = ["SKILLED NURSING FACILITY", 'REHAB', 'CHRONIC/LONG TERM ACUTE CARE', 'PSYCH FACILITY', 'ACUTE HOSPITAL', 'OTHER FACILITY', 'HEALTHCARE FACILITY']
hosp['admissions']['discharged_skilled_facility'] = hosp['admissions']['discharge_location'].apply(lambda x: x in options).astype(int)

# Then select the desired columns to form your new DataFrame.
discharge_locations = hosp['admissions'][['hadm_id', 'discharge_location', 'discharged_home', 'discharged_hospice', 'discharged_skilled_facility']]

## 15. Death within 30 days

In [ ]:
# Admission time
hosp['admissions']['admittime'] = pd.to_datetime(hosp['admissions']['admittime'])
hosp['admissions']

# Date of death for each subject
df_deaths = hosp['patients'][['subject_id', 'dod']]

# New tables of subject id, hospitalisation id, admission times and deaths
df_details = hosp['admissions'][['subject_id', 'hadm_id', 'admittime', 'dischtime']]

# Combine this into one df
deathin30_df = df_details.merge(df_deaths, left_on = "subject_id", right_on = "subject_id")

# Set as a ts and edit the dates so that each date is midday
deathin30_df['dod'] = pd.to_datetime(deathin30_df['dod'])
deathin30_df['dod'] = deathin30_df['dod'].dt.normalize() + pd.Timedelta(hours=12)

# Create a new colums if they died within 30 days of admission
deathin30_df['time2death'] = deathin30_df['dod'] - deathin30_df['admittime']
deathin30_df['deathin30'] = deathin30_df['time2death'].apply(lambda x: x < pd.Timedelta(days=30) if pd.notnull(x) else False).astype(int)

# Sort by subject_id and then admittime
deathin30_df.sort_values(['subject_id', 'admittime'], inplace = True)

## 16. Hospital readmission within 30 days
This should be calculated for each hospitalisation. First, get a list of all hospitalisation admissions. For each hospitalisation find out if there was a subsequent admission and, if so, when it was. Then calculate the time between discharge and subsequent admission.

In [ ]:
hosp['admissions'] = hosp['admissions'].sort_values(['subject_id', 'admittime'])
hosp['admissions']['next_admission_date'] = hosp['admissions'].groupby('subject_id')['admittime'].shift(-1)

# Create a new colums for readmission
hosp['admissions']['readmission_time'] = hosp['admissions']['next_admission_date'] - hosp['admissions']['dischtime']
hosp['admissions']['readmissionin30'] = hosp['admissions']['readmission_time'].apply(lambda x: x < pd.Timedelta(days=30) if pd.notnull(x) else False).astype(int)

# Boil it down to relevant columns
readmission_df = hosp['admissions'][['subject_id', 'hadm_id', 'admittime', 'dischtime', 'readmission_time', 'readmissionin30']]

## [6. From earlier] Severity of illness at hospital admission
The LAPS2 Score. NOTE: LASP2 took a lot of calculation and is therefore toggled. The code is roughly a similar length to the rest of the notebook.

### Calculating LAPS2

### Set up

We recode all of the datasets so that the stay_ids are mapped to the first stay_id of that hospitalisation. This way all of the icu stays in each hospitalisation are viewed as a single hospitalisatoin and we can base it around hospitalisations not ICU stays. This will help reframe the problem in terms of hospitalisations and make sure the data is as close to hospital admission as possible.


In [ ]:
# Read in the Glasgow Coma Scale (gcs) data. This is from the derived section of the GoogleBigQuery database. NOTE: it is not the same as first_day_gcs
gcs = pd.read_csv("data/queried_data/gcs.csv")

# Create a feature for gender
gender = hosp['patients'][['subject_id', 'gender']].copy()
gender['gender'] = (gender['gender'] == 'M').astype(int)

In [ ]:
# Create a copy of icu stays as about to perturb it
icustays_individuals = icu['icustays'][['subject_id', 'hadm_id', 'stay_id']].copy()

In [ ]:
# Define a base icu dataset
base = icu['icustays'][['hadm_id', 'stay_id', 'intime']].copy()

# Define intime as a datatime
base['intime'] = pd.to_datetime(base['intime'])

# Sort by hadm_id and then by intime so that the first admissions are on top
base.sort_values(['hadm_id', 'intime'], inplace = True)

# Remove duplicates and keep the first to get the mapping. 
base_short = base.drop_duplicates(subset='hadm_id', keep='first').copy()

# Create a mapping between hospitalions and stay_id (this is the first stay)
first_icu_mapping = base_short.set_index('hadm_id')['stay_id'].to_dict()

# Go back to base and add a new column which is the first stay_id
base['first_icu'] = base['hadm_id'].apply(lambda x: first_icu_mapping[x])

# Create another dict of the final mapping.
stay_id_map = base.set_index('hadm_id')['first_icu'].to_dict()

# Create final mapping (for datasets without hosp)
original_stay_id2first_id = base.set_index('stay_id')['first_icu'].to_dict()

In [ ]:
# dfs to transform (with and without hosp)
dataframes_to_transform_1 = [icustays_individuals, icu['ingredientevents'], icu['inputevents'], icu['procedureevents']]
dataframes_to_transform_2 = [icu['chartevents'], gcs]

# Apply the mapping
for df in dataframes_to_transform_1:
    df['stay_id'] = df['hadm_id'].map(stay_id_map)

for df in dataframes_to_transform_2:
    df['stay_id'] = df['stay_id'].map(original_stay_id2first_id)

In [ ]:
# Cut icustays down. This avoid duplicates being added.
icustays_individuals = icustays_individuals.drop_duplicates(keep='first').copy()

### Define functions to collect the first reading of each hospitalisation. Need different collectors for different datasets

In [ ]:
#### A function to collect the first reading of each hospitalisation
def hosp_lab_collector(labels, details = False):

    # Return ids
    labels_ids = hosp['d_labitems'][hosp['d_labitems'].label.isin(labels)].itemid
    if details == True:
        print(hosp['d_labitems'][hosp['d_labitems'].label.isin(labels)])

    # Make smaller table
    table = hosp['labevents'][hosp['labevents'].itemid.isin(labels_ids)].copy()
    if details == True:
        print(table.itemid.value_counts())

    # Record '___' as pd.NA
    #display(table)
    table = table.replace('___', pd.NA)

    # Create an auxiliary column
    table['is_value_na'] = table['valuenum'].isnull()

    # Sort by hadm_id, then by whether the value is null, and then by charttime. This is neats. Avoids NAs unless it has to!
    table.sort_values(['hadm_id', 'is_value_na', 'charttime'], inplace = True)

    # Remove duplicates of hadm_id (keep the first)
    table = table.drop_duplicates(subset='hadm_id', keep='first')

    return table

In [ ]:
#### A function to collect the first reading of each hospitalisation
def icu_chart_collector_secondary(labels, details = False, item_ids = []):
    """
    The original ICU collector. For use in the prelim round and in the secondary round if appropriate
    This works across secondary (uses the secondary dataset)
    """

    # Return ids
    labels_ids = icu['d_items'][icu['d_items'].label.isin(labels)].itemid

    if len(item_ids) > 0:
        labels_ids = item_ids

    if details == True:
        display(icu['d_items'][icu['d_items'].label.isin(labels)])

    # Make smaller table from icu['chartevents']
    table_icu = icu['chartevents'][icu['chartevents'].itemid.isin(labels_ids)].copy()
    if details == True:
        display(table_icu)
        display(table_icu.itemid.value_counts())

    # Switch relevant values to NA
    table_icu = table_icu.replace('___', pd.NA)

    # Create an auxiliary column
    table_icu['is_value_na'] = table_icu['valuenum'].isnull()

    # Sort by stay_id, then by whether the value is null, and then by charttime. Avoids NAs unless it has to!
    table_icu.sort_values(['stay_id', 'is_value_na', 'charttime'], inplace = True)

    # Remove duplicates of stay_id (keep the first)
    table_icu = table_icu.drop_duplicates(subset='stay_id', keep='first')

    return table_icu

In [ ]:
# Define the secondary model's feature collector. Note that I use the prelim model ICU collector.
def hosp_lab_collector_secondary(labels, worse, details = False, item_ids = []):
    """
    Added a time filter. Only includes those within 72 hours of hospital admission. 
    If multiple values returns the most medically severe value. Min / Max needs to be defined
    """

    # Return ids
    labels_ids = hosp['d_labitems'][hosp['d_labitems'].label.isin(labels)].itemid

    if len(item_ids) > 0:
        labels_ids = item_ids

    if details == True:
        print(hosp['d_labitems'][hosp['d_labitems'].label.isin(labels)])

    # Make smaller table from secondary_labevents
    table = hosp['labevents'][hosp['labevents'].itemid.isin(labels_ids)].copy()
    if details == True:
        print(table.itemid.value_counts())

    #### TIME FILTER ####

    # TIME FILTER: Append hospital admission time. 
    table = table.merge(hosp['admissions'][['hadm_id', 'admittime']], on="hadm_id", how='left')

    # TIME FILTER: Make things date time
    table['admittime'] = pd.to_datetime(table['admittime'])
    table['charttime'] = pd.to_datetime(table['charttime'])

    # TIME FILTER: Make a time after admission col and filter to only those in 72 hours of admission
    table['time_after_admission'] = table['charttime'] - table['admittime']
    table  =  table[table['time_after_admission'] < pd.Timedelta(hours=72)]

    #### DEAL WITH MULTIPLE VALUES ####

    # Record '___' as pd.NA
    table = table.replace('___', pd.NA)
    
    # Create an auxiliary column
    table['is_value_na'] = table['valuenum'].isnull()

    if worse == 'higher':
        table.sort_values(by=['hadm_id', 'is_value_na', 'valuenum'], ascending=[True, True, False], inplace=True)

    if worse == 'lower':
        table.sort_values(by=['hadm_id', 'is_value_na', 'valuenum'], ascending=[True, True, True], inplace=True)

    # Remove duplicates of hadm_id (keep the first)
    table = table.drop_duplicates(subset='hadm_id', keep='first')

    return table

### Preliminary model

In [ ]:
#### Collect BUN Manually #### itemid = 225624

# Define the labels. Removed ['BUN_ApacheIV', 'BunScore_ApacheIV'] as scores seem a little different. Only 8 of 227001 and 227000 (the rest are BUN)
bun_icu = ['BUN']

# Table of items that suffice as BUN
bun_icu_itemids = icu['d_items'][icu['d_items'].label.isin(bun_icu)].itemid

# Chartevents filtered by BUN. 
chartevents_bun = icu['chartevents'][icu['chartevents'].itemid.isin(bun_icu_itemids)].copy()

# Make charttime a time
chartevents_bun['charttime'] = pd.to_datetime(chartevents_bun['charttime'])

# Record '___' as pd.NA
chartevents_bun = chartevents_bun.replace('___', pd.NA)

# Create an auxiliary column
chartevents_bun['is_value_na'] = chartevents_bun['valuenum'].isnull()

# Sort by stay_id and then charttime
chartevents_bun.sort_values(['stay_id', 'is_value_na', 'charttime'], inplace = True)

# Remove duplicates of stay_id (keep the first)
chartevents_bun = chartevents_bun.drop_duplicates(subset='stay_id', keep='first')

# Rename and export
chartevents_bun.rename(columns = {'valuenum' : 'BUN'}, inplace = True)
chartevents_bun = chartevents_bun[['stay_id', 'BUN']]

In [ ]:
# Creatinine details 50912, 51081, 51977, 52024, 52546
Creatinine_hosp = ['Creatinine', 'Creatinine, Whole Blood', 'Creatinine, Serum', 'Creatinine, Blood']
Creatinine_labevents = hosp_lab_collector(Creatinine_hosp, details = False)[['hadm_id', 'valuenum']]
Creatinine_labevents.rename(columns = {'valuenum':'creatinine'}, inplace = True)

#### SODIUM #### 50983 (Sodium or Sodium [Moles/volume] in Serum or Plasma). 50824, 50983, 52455, 52623
#  50824 (Sodium, Whole Blood). All measured in mEq/L
Sodium_hosp = ['Sodium', 'Sodium, Whole Blood']
Sodium_labevents = hosp_lab_collector(Sodium_hosp, details = False)[['hadm_id', 'valuenum']]
Sodium_labevents.rename(columns = {'valuenum':'sodium'}, inplace = True)

#### ANION GAP #### 52500 and 50868 (both Anion gap in Serum or Plasma). Both mEq/L. All 50868 (Anion gap 4 in Serum or Plasma). 50868, 52500
ANION_hosp = ['Anion Gap']
ANION_labevents = hosp_lab_collector(ANION_hosp, details = False)[['hadm_id', 'valuenum']]
ANION_labevents.rename(columns = {'valuenum':'anion_gap'}, inplace = True)

#### Serum Bicarbonate #### Almsot all 50882 (mEq/L) (Bicarbonate [Moles/volume] in Serum or Plasma). 50803, 50882, 52039
Bicarbonate_hosp = ['Bicarbonate', 'Calculated Bicarbonate, Whole Blood', 'Calculated Bicarbonate']
Bicarbonate_labevents = hosp_lab_collector(Bicarbonate_hosp, details = False)[['hadm_id', 'valuenum']]
Bicarbonate_labevents.rename(columns = {'valuenum':'Bicarbonate'}, inplace = True)

In [ ]:
### Prelim Model Dataset ###
stays = icustays_individuals[['subject_id', 'hadm_id', 'stay_id']]
stays = stays.merge(Creatinine_labevents, on="hadm_id", how='left')
stays = stays.merge(chartevents_bun, on="stay_id", how='left')
stays = stays.merge(Bicarbonate_labevents, on="hadm_id", how='left')
stays = stays.merge(ANION_labevents, on="hadm_id", how='left')
stays = stays.merge(Sodium_labevents, on="hadm_id", how='left')
stays = stays.merge(derived_age, on="hadm_id", how='left')
stays = stays.merge(gender, on="subject_id", how='left')
stays = stays.merge(admission_measures[['hadm_id', 'ED_Admission']], on="hadm_id", how='left') 

In [ ]:
# Make floats
for col in ['creatinine', 'BUN', 'Bicarbonate', 'anion_gap','sodium']:
    stays[col] = pd.to_numeric(stays[col], errors = 'coerce')

# Remove extreme outliers. Define upper limits for each lab value
upper_limits = {
    'creatinine': 3,  
    'BUN': 100.0,  
    'Bicarbonate': 50.0,  
    'anion_gap': 30.0, 
    'sodium': 200.0
}

# Replace outliers with NA
for col, upper_limit in upper_limits.items():
    stays.loc[stays[col] > upper_limit, col] = np.nan

# Define representative values.
representative_values = {
    'creatinine': 1.0,
    'BUN': 14.0,
    'Bicarbonate': 26.0,
    'anion_gap': 7.0, 
    'sodium': 140.0
}

# Impute missing values
for col, representative_value in representative_values.items():
    stays[col].fillna(representative_value, inplace=True)

# Make ratios
stays['BUN/creatinine'] = stays['BUN'] / stays['creatinine']
stays['AnionGap/SerumBicarbonate'] = (stays['anion_gap'] / stays['Bicarbonate']) *1000

### Make all of the dummies ###

def create_dummies(df, column, bins):
    # Create binned categories
    categories = pd.cut(df[column], bins, labels=False, include_lowest=True, right=False)

    # Create dummies
    dummies = pd.get_dummies(categories, prefix=column)

    # Drop the original column from the DataFrame and add the dummies
    df = pd.concat([df, dummies], axis=1)
    
    return df

# Define the bins. Note that if you are on the edge you move into the upper bin.
age_bins = [18, 39, 64, 74, 84, float('inf')]  # 'inf' for ages 85 and above. Reference 18 - 39 (age_0)
BUN_creatinine_bins = [0, 8, 16, 24, float('inf')] # Reference 8 - 16 BUN/creatinine_1
sodium_bins = [0, 129, 132, 135, 146, 149, 155, float('inf')] # Reference 135 – 146, sodium_3
anion_gap_bicarbonate_bins = [0, 200, 400, 600, float('inf')] # Reference 200 - 400, AnionGap/SerumBicarbonate_1

# Apply the function
stays = create_dummies(stays, 'age', age_bins)
stays = create_dummies(stays, 'BUN/creatinine', BUN_creatinine_bins)
stays = create_dummies(stays, 'sodium', sodium_bins)
stays = create_dummies(stays, 'AnionGap/SerumBicarbonate', anion_gap_bicarbonate_bins)

stays_original = stays.copy()

In [ ]:
# Only keep necessary columns
stays_trim = stays[['stay_id', 'age_1', 'age_2', 'age_3', 'age_4', 'gender', 'ED_Admission',
  'BUN/creatinine_0.0', 'BUN/creatinine_2.0', 'BUN/creatinine_3.0', 'sodium_0', 'sodium_1', 'sodium_2'
 , 'sodium_4', 'sodium_5', 'sodium_6', 'AnionGap/SerumBicarbonate_0.0', 'AnionGap/SerumBicarbonate_2.0',
 'AnionGap/SerumBicarbonate_3.0']].copy()

 # Make the coefficient dictionary
coeffs = {
    'age_1': -0.25234, 
    'age_2': 0.25894, 
    'age_3': 0.48826, 
    'age_4': 0.87647, 
    'gender': 0.27430, 
    'ED_Admission': 1.39670,
    'BUN/creatinine_0.0': 0.26988, 
    'BUN/creatinine_2.0': -0.22465, 
    'BUN/creatinine_3.0': 0.39858, 
    'sodium_0': 0.11980, 
    'sodium_1': -0.06801, 
    'sodium_2': -0.30494,
    'sodium_4': -0.02560, 
    'sodium_5': 0.42071, 
    'sodium_6': 0.58891, 
    'AnionGap/SerumBicarbonate_0.0': -0.20038, 
    'AnionGap/SerumBicarbonate_2.0': -0.11174,
    'AnionGap/SerumBicarbonate_3.0': 0.70227
}

intercept = -4.31678

In [ ]:
# Prepare features and coefficients in the same order
features = list(coeffs.keys())
coefficients = np.array(list(coeffs.values()))

# Apply the logistic regression formula
linear_combination = np.dot(stays_trim[features], coefficients) + intercept
l = [-i for i in linear_combination] # Overcome weird bug. Confirmed that this gets the correct output
probabilities = 1 / (1 + np.exp(l))

# Add the probs to the stays original
stays_original['probs'] = probabilities

# Have a columns indicating whether they are high-risk of not (i.e. prob > 0.06)
stays_original['high_risk'] = (stays_original['probs']>0.06).astype(int)

### Secondary model

#### Feature creation

In [ ]:
# Codes needed

# 50820, 52041, 50831, 51094, 51491, 52730, 50813, 52442, 53154, 50824, 50983, 52455, 52623, 50885, 50912, 51081, 51977, 52024, 52546
# 50862, 51775, 51776, 51807, 51836, 51910, 51926, 52022, 53085, 53116, 53138, 50809, 50931, 51478, 51981, 52027, 52569, 51221, 51480
# 51638, 51639, 52028, 51300, 51301, 51755, 51756, 53134, 50818, 52040, 50821, 52042, 51003, 50825

# 225624, 227000, 227001, 228152, 229669, 220227, 226767, 226860, 226861, 226862, 226863, 226865, 227035, 228232, 220179, 225309, 226850
# 223761, 223762, 220045, 220210, 224688, 224690, 224745, 225475, 227032, 227047, 229425, 229563, 220050, 220052, 

In [ ]:
#### Arterial pH ####
Arterial_pH = ['pH', 'pH ', 'pH  ']
Arterial_pH_labevents = hosp_lab_collector_secondary(Arterial_pH, worse = 'lower', details = False, item_ids = [50820, 52041])[['hadm_id', 'valuenum']]
Arterial_pH_labevents.rename(columns = {'valuenum':'Arterial_pH'}, inplace = True)

#### Lactate #### 
Lactate_hosp = ['Lactate'] # Just want this. Measrued in mmol/L so okay.
Lactate_labevents = hosp_lab_collector_secondary(Lactate_hosp, worse = 'higher', details = False)[['hadm_id', 'valuenum']]
Lactate_labevents.rename(columns = {'valuenum':'Lactate'}, inplace = True)

#### SODIUM #### 
#  50983 (Sodium or Sodium [Moles/volume] in Serum or Plasma). Same unit as mEq/L for sodium
#  50824 (Sodium, Whole Blood). All measured in mEq/L
Sodium_hosp = ['Sodium', 'Sodium, Whole Blood']
Sodium_labevents = hosp_lab_collector_secondary(Sodium_hosp, worse = 'lower', details = False)[['hadm_id', 'valuenum']]
Sodium_labevents.rename(columns = {'valuenum':'sodium'}, inplace = True)

#### TOTAL SERUM BILIRUBIN #### 50885
BILIRUBIN_hosp = []
BILIRUBIN_labevents = hosp_lab_collector_secondary(BILIRUBIN_hosp, worse = 'higher', details = False, item_ids=[50885])[['hadm_id', 'valuenum']]
BILIRUBIN_labevents.rename(columns = {'valuenum':'Biblirubin'}, inplace = True)

#### BUN #### 
BUN_icu = ['BUN', 'BUN_ApacheIV', 'BunScore_ApacheIV']
BUN_events = icu_chart_collector_secondary(BUN_icu, details = False)[['stay_id', 'valuenum']]
BUN_events.rename(columns = {'valuenum':'BUN'}, inplace = True)

# Creatinine details
Creatinine_hosp = ['Creatinine', 'Creatinine, Whole Blood', 'Creatinine, Serum', 'Creatinine, Blood']
Creatinine_labevents = hosp_lab_collector_secondary(Creatinine_hosp, worse = 'higher', details = False, item_ids=[50912, 52024, 52546])[['hadm_id', 'valuenum']]
Creatinine_labevents.rename(columns = {'valuenum':'creatinine'}, inplace = True)

# Change 0s to NA (will get the same points in the scoring)
Creatinine_labevents['creatinine'].replace(0.0, pd.NA, inplace = True)

# Albumin details
Albumin_hosp = ['Albumin, Blood', '(Albumin)', 'Albumin', '<Albumin>', '(Albumin) ', 'Albumin ']
Albumin_labevents = hosp_lab_collector_secondary(Albumin_hosp, worse = 'lower', details = False, item_ids=[50862, 52022])[['hadm_id', 'valuenum']]
Albumin_labevents.rename(columns = {'valuenum':'albumin'}, inplace = True)

# Glucose details
Glucose_hosp = ['Glucose, Whole Blood', 'Glucose', 'Glucose ']
Glucose_labevents = hosp_lab_collector_secondary(Glucose_hosp, worse = 'lower', details = False, item_ids=[50931, 50809, 52027, 52569])[['hadm_id', 'valuenum']]
Glucose_labevents.rename(columns = {'valuenum':'glucose'}, inplace = True)

# Hematocrit details. All 51221
Hematocrit_hosp = ['Hematocrit', 'Hematocrit ', 'Hematocrit  ']
Hematocrit_labevents = hosp_lab_collector_secondary(Hematocrit_hosp, worse = 'lower', details = False, item_ids=[51221, 51638, 51639, 52028])[['hadm_id', 'valuenum']]
Hematocrit_labevents.rename(columns = {'valuenum':'hematocrit'}, inplace = True)

# WBC. 51301 or 51300 but all work
white_blood_cells_hosp = ['White Blood Cells', 'Absolute Other WBC', 'WBC Count']
white_blood_cells_labevents = hosp_lab_collector_secondary(white_blood_cells_hosp, worse = 'higher', details = False, item_ids=[])[['hadm_id', 'valuenum']]
white_blood_cells_labevents.rename(columns = {'valuenum':'white_blood_cells'}, inplace = True)

# ARTERIAL PaCO2 (mm Hg).
pCO2_hosp = ['pCO2', 'pCO2 ']
pCO2_labevents = hosp_lab_collector_secondary(pCO2_hosp, worse = 'higher', details = False, item_ids=[])[['hadm_id', 'valuenum']]
pCO2_labevents.rename(columns = {'valuenum':'pCO2'}, inplace = True)

# ARTERIAL PaO2 (mm Hg).
pO2_hosp = ['pO2', 'pO2 ']
pO2_labevents = hosp_lab_collector_secondary(pO2_hosp, worse = 'higher', details = False, item_ids=[])[['hadm_id', 'valuenum']]
pO2_labevents.rename(columns = {'valuenum':'pO2'}, inplace = True)

In [ ]:
# Troponin T

Troponin_labels = ['Troponin T']
# Return ids
Troponin_labels_ids = hosp['d_labitems'][hosp['d_labitems'].label.isin(Troponin_labels)].itemid

# Make smaller table
Troponin_table = hosp['labevents'][hosp['labevents'].itemid.isin(Troponin_labels_ids)].copy()

#### TIME FILTER ####

# TIME FILTER: Append hospital admission time. 
Troponin_table = Troponin_table.merge(hosp['admissions'][['hadm_id', 'admittime']], on="hadm_id", how='left')

# TIME FILTER: Make things date time
Troponin_table['admittime'] = pd.to_datetime(Troponin_table['admittime'])
Troponin_table['charttime'] = pd.to_datetime(Troponin_table['charttime'])

# TIME FILTER: Make a time after admission col and filter to only those in 72 hours of admission
Troponin_table['time_after_admission'] = Troponin_table['charttime'] - Troponin_table['admittime']
Troponin_table  =  Troponin_table[Troponin_table['time_after_admission'] < pd.Timedelta(hours=72)]

#### DEAL WITH MULTIPLE VALUES ####

# Record '___' as pd.NA
Troponin_table = Troponin_table.replace('___', pd.NA)

# Create an auxiliary column
Troponin_table['is_value_na'] = Troponin_table['valuenum'].isnull()

Troponin_table.sort_values(by=['hadm_id', 'is_value_na', 'valuenum'], ascending=[True, True, False], inplace=True)

# Remove duplicates of hadm_id (keep the first)
Troponin_table = Troponin_table.drop_duplicates(subset='hadm_id', keep='first')

Troponin_table = Troponin_table[['hadm_id', 'valuenum']]
Troponin_table.rename(columns = {'valuenum':'Troponin_T'}, inplace = True)

In [ ]:
# Temperature
Temperature_hosp = ['Temperature']
Temperature_labevents = hosp_lab_collector_secondary(Temperature_hosp, worse = 'lower', details = False, item_ids=[])[['hadm_id', 'valuenum']]
Temperature_labevents.rename(columns = {'valuenum':'temperature'}, inplace = True)

# Temperature (FROM ICU - preferred). Note that custom function used to convert C ---> F
temp_labels = ['Temperature Fahrenheit', 'Temperature Celsius'] # 223761, 223762

# Return ids
temp_labels_ids = icu['d_items'][icu['d_items'].label.isin(temp_labels)].itemid

# Make smaller table
temp_table = icu['chartevents'][icu['chartevents'].itemid.isin(temp_labels_ids)].copy()

# Switch relevant values to NA
temp_table = temp_table.replace('___', pd.NA)

# Convert C into F
temp_table['valuenum'] = pd.to_numeric(temp_table['valuenum'], errors = 'coerce')
temp_table['valuenum'] = temp_table.apply(lambda row: ((row['valuenum'] * 9/5) + 32) if row['itemid'] == 223762 else row['valuenum'], axis=1)

# Create an auxiliary column
temp_table['is_value_na'] = temp_table['valuenum'].isnull()

# Sort by stay_id, then by whether the value is null, and then by charttime.
temp_table.sort_values(['stay_id', 'is_value_na', 'charttime'], inplace = True)

# Remove duplicates of stay_id (keep the first)
temp_table = temp_table.drop_duplicates(subset='stay_id', keep='first')
temp_table.rename(columns = {'valuenum':'temperature'}, inplace = True)

In [ ]:
# Heart Rate (beats per minute) 220045 (Heart Rate)
HR_hosp = ['Heart Rate']
HR_labevents = icu_chart_collector_secondary(HR_hosp, details = False, item_ids=[])[['stay_id', 'valuenum']]
HR_labevents.rename(columns = {'valuenum':'HR'}, inplace = True)

# RR [ICU]. 220210 (Respiratory Rate), 224690 Respiratory Rate (Total), 224688
RR_ICU = ['Plan-Respiratory', 'ROS-Respiratory', 'Post-Operative Respiratory (RESPIRAT)', 'Non-Operative Respiratory (RESPIRAT)',
'Respiratory Rate (Set)', 'Respiratory Quotient', 'Respiratory Rate (Total)', 'Respiratory Rate',
 'Respiratory Arrest']
RR_ICU_EVENTS = icu_chart_collector_secondary(RR_ICU, details = False, item_ids=[220210, 224690, 224688])[['stay_id', 'valuenum']]
RR_ICU_EVENTS.rename(columns = {'valuenum':'RR'}, inplace = True)

# Systolic Blood Pressure 220179 (Non Invasive Blood Pressure systolic), 220052 (PA systolic pressure(PA Line))
# 220050 (Arterial Blood Pressure systolic). 225309 (ART BP Systolic). 220059 (Pulmonary Artery Pressure systolic)
Blood_pressure_ICU = ['Pulmonary Atrtery Pressure Signal - Systolic',
 'Aortic Pressure Signal - Systolic',
 'RV systolic pressure(PA Line)', 'ART BP Systolic',
 'Non Invasive Blood Pressure systolic', 'Arterial Blood Pressure systolic', 
  'Arterial Blood Pressure mean']
Blood_pressure = icu_chart_collector_secondary(Blood_pressure_ICU, details = False, item_ids=[220179, 220050, 225309])[['stay_id', 'valuenum']]
Blood_pressure.rename(columns = {'valuenum':'blood_pressure'}, inplace = True)

# Change 0s to 1s (will get the same points in the scoring)
Blood_pressure['blood_pressure'].replace(0.0, 1, inplace = True)

# Oxygend Saturation 220277 is SpO2 - this is the ideal. 220227 is SaO2 (a bit different).
Oxygen_ICU = ['PAR-Oxygen saturation', 
 'OxygenApacheIIScore',
 'OxygenScore_ApacheIV', 'Arterial O2 Saturation',
 'O2 saturation pulseoxymetry', 'PA %O2 Saturation (PA Line)', 
  'PVR %O2 Saturation (PA Line)', 'ART %O2 saturation (PA Line)', 'SVR %O2 Saturation (PA Line)', 'RA %O2 Saturation (PA Line)']
Oxygen_pressure = icu_chart_collector_secondary(Oxygen_ICU, details = False, item_ids=[220277, 220227])[['stay_id', 'valuenum']]
Oxygen_pressure.rename(columns = {'valuenum':'oxygen'}, inplace = True)

In [ ]:
# Neurological Score (GCS) ---> Separate Derived Table

# Record '___' as pd.NA
gcs_table = gcs.replace('___', pd.NA)

# Create an auxiliary column
gcs_table['is_value_na'] = gcs_table['gcs'].isnull()

# Define charttime as a time
gcs_table['charttime'] = pd.to_datetime(gcs_table['charttime'])
gcs_table.sort_values(['stay_id', 'charttime'], inplace = True)

# Remove duplicates of stay_id (keep the first)
gcs_table = gcs_table.drop_duplicates(subset='stay_id', keep='first')

# Dictionary of values
gcs_dict = {15:"Normal", 14:"Abmiguous", 13:"Abmiguous",
12:"Abnormal", 11:"Abnormal", 10:"Abnormal", 9:"Abnormal",
8:"Very Abnormal", 7:"Very Abnormal", 6:"Very Abnormal", 5:"Very Abnormal",
4:"Very Abnormal", 3:"Very Abnormal", 2:"Very Abnormal", 1:"Very Abnormal", 0:"Very Abnormal"}

gcs_table['Neurological_Score'] = gcs_table["gcs"].apply(lambda x: gcs_dict[x])
gcs_table = gcs_table[["stay_id", "Neurological_Score"]]

#### Putting it all together

In [ ]:
# Make a base
LAPS2_DF = icustays_individuals[['subject_id', 'hadm_id', 'stay_id']]

# Merge in the high risk category 
LAPS2_DF = LAPS2_DF.merge(stays_original[['stay_id', 'high_risk']], on="stay_id", how='left')

# Merge in actual death for checks and for the model
death = hosp['admissions'][['hospital_expire_flag', 'hadm_id']].copy()
death.rename(columns = {'hospital_expire_flag':'death'}, inplace = True)
LAPS2_DF = LAPS2_DF.merge(death, on="hadm_id", how='left')

# Merge in death within 30 days of admission deathin30_df
deathin30 = deathin30_df[['hadm_id', 'deathin30']].copy()
LAPS2_DF = LAPS2_DF.merge(deathin30, on="hadm_id", how='left')

# Merge everything in
LAPS2_DF = LAPS2_DF.merge(Arterial_pH_labevents, on="hadm_id", how='left')
LAPS2_DF = LAPS2_DF.merge(Lactate_labevents, on="hadm_id", how='left')
LAPS2_DF = LAPS2_DF.merge(Sodium_labevents, on="hadm_id", how='left')
LAPS2_DF = LAPS2_DF.merge(BILIRUBIN_labevents, on="hadm_id", how='left')
LAPS2_DF = LAPS2_DF.merge(BUN_events, on="stay_id", how='left')
LAPS2_DF = LAPS2_DF.merge(Creatinine_labevents, on="hadm_id", how='left')

# Make BUN_creatinine
LAPS2_DF['BUN_creatinine'] = LAPS2_DF['BUN'] / LAPS2_DF['creatinine']

LAPS2_DF = LAPS2_DF.merge(Albumin_labevents, on="hadm_id", how='left') 
LAPS2_DF = LAPS2_DF.merge(Glucose_labevents, on="hadm_id", how='left') 
LAPS2_DF = LAPS2_DF.merge(Hematocrit_labevents, on="hadm_id", how='left') 
LAPS2_DF = LAPS2_DF.merge(white_blood_cells_labevents, on="hadm_id", how='left') 
LAPS2_DF = LAPS2_DF.merge(pCO2_labevents, on="hadm_id", how='left') 
LAPS2_DF = LAPS2_DF.merge(pO2_labevents, on="hadm_id", how='left') 
LAPS2_DF = LAPS2_DF.merge(Troponin_table, on="hadm_id", how='left') 
LAPS2_DF = LAPS2_DF.merge(temp_table[['stay_id', 'temperature']], on="stay_id", how='left') 
LAPS2_DF = LAPS2_DF.merge(HR_labevents, on="stay_id", how='left') 
LAPS2_DF = LAPS2_DF.merge(RR_ICU_EVENTS, on="stay_id", how='left') 
LAPS2_DF = LAPS2_DF.merge(Blood_pressure, on="stay_id", how='left') 

# Make shock_index
LAPS2_DF['shock_index'] = LAPS2_DF['HR'] / LAPS2_DF['blood_pressure']

LAPS2_DF = LAPS2_DF.merge(Oxygen_pressure, on="stay_id", how='left')
LAPS2_DF = LAPS2_DF.merge(gcs_table, on="stay_id", how='left') 

# Calculate missing data across each row
LAPS2_DF['missing_counts'] = LAPS2_DF.isnull().sum(axis=1)
LAPS2_DF['more_than_half_missing'] = (LAPS2_DF['missing_counts']>10).astype(int)

#### Points creator

In [ ]:
def points_assigner(df, column_name, splits, points_dict, missingness_points, high_risk_missingness_points):
    """
    NOTE: The missingness and high_risk_missingness are the values to fill in
    """
    column = df[column_name].copy()
    column = pd.to_numeric(column, errors = 'coerce')
    high_risk_col = df['high_risk']

    # Assign bin labels to each
    bin_labels = range(len(splits)-1)
    column = pd.cut(column, bins=splits, labels=bin_labels, include_lowest=True, right=False)
    column = column.cat.codes

    # Convert this into points
    column = column.map(points_dict)

    # Assign high_risk_missingness to NA values where high_risk == 1. Fill in the points for high risk missingess
    column.loc[(column.isnull()) & (high_risk_col == 1)] = high_risk_missingness_points

    # Assign missingness to remaining NA values. Fill in the points for missingess
    column.fillna(missingness_points, inplace=True)

    return column

#### Assign the points

In [ ]:
# Points Data Frame
Points = LAPS2_DF[['hadm_id', 'stay_id', 'high_risk', 'death', 'missing_counts', 'more_than_half_missing', 'Arterial_pH', 'Lactate']].copy()

In [ ]:
# Arterial_pH Lactate
for index, row in LAPS2_DF.iterrows():
    if (row['Arterial_pH'] < 7.20) and (row['Lactate'] <= 2.00):
        Points.loc[index, 'Arterial_pH_Lactate'] = 13  # replace 'your_value' with the value you want to assign
    
    elif (row['Arterial_pH'] < 7.20) and (row['Lactate'] < 4.00):
        Points.loc[index, 'Arterial_pH_Lactate'] = 19 

    elif (row['Arterial_pH'] < 7.20) and (row['Lactate'] >= 4.00):
        Points.loc[index, 'Arterial_pH_Lactate'] = 34

    elif (row['Arterial_pH'] < 7.35) and (row['Lactate'] <= 2.00):
        Points.loc[index, 'Arterial_pH_Lactate'] = 5  

    elif (row['Arterial_pH'] < 7.35) and (row['Lactate'] < 4.00):
        Points.loc[index, 'Arterial_pH_Lactate'] = 15  

    elif (row['Arterial_pH'] < 7.35) and (row['Lactate'] >= 4.00):
        Points.loc[index, 'Arterial_pH_Lactate'] = 25  

    elif (row['Arterial_pH'] < 7.45) and (row['Lactate'] <= 2.00):
        Points.loc[index, 'Arterial_pH_Lactate'] = 0  

    elif (row['Arterial_pH'] < 7.45) and (row['Lactate'] < 4.00):
        Points.loc[index, 'Arterial_pH_Lactate'] = 12  

    elif (row['Arterial_pH'] < 7.45) and (row['Lactate'] >= 4.00):
        Points.loc[index, 'Arterial_pH_Lactate'] = 26  

    elif (row['Arterial_pH'] >= 7.45) and (row['Lactate'] <= 2.00):
        Points.loc[index, 'Arterial_pH_Lactate'] = 12  

    elif (row['Arterial_pH'] >= 7.45) and (row['Lactate'] < 4.00):
        Points.loc[index, 'Arterial_pH_Lactate'] = 15 

    elif (row['Arterial_pH'] >= 7.45) and (row['Lactate'] >= 4.00):
        Points.loc[index, 'Arterial_pH_Lactate'] = 30 

# Fill the rest with 0
Points['Arterial_pH_Lactate'].fillna(0, inplace=True)

In [ ]:
# sodium
sodium_splits = [0, 5, 13, np.inf]
sodium_points = {0:8, 1:0, 2:11}

# Apply the points assigner
Points['sodium'] = points_assigner(df = LAPS2_DF, column_name = 'sodium', splits = sodium_splits,
 points_dict = sodium_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# Biblirubin
Biblirubin_splits = [0, 2, 3, 5, 8, np.inf]
Biblirubin_points = {0:0, 1:11, 2:18, 3:25, 4:41}

# Apply the points assigner
Points['Biblirubin'] = points_assigner(df = LAPS2_DF, column_name = 'Biblirubin', splits = Biblirubin_splits,
 points_dict = Biblirubin_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# BUN
BUN_splits = [0, 18, 20, 40, 80, np.inf]
BUN_points = {0:0, 1:11, 2:12, 3:20, 4:25}

# Apply the points assigner
Points['BUN'] = points_assigner(df = LAPS2_DF, column_name = 'BUN', splits = BUN_splits,
 points_dict = BUN_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# creatinine
creatinine_splits = [0, 1, 2, 4, np.inf]
creatinine_points = {0:0, 1:6, 2:11, 3:5}

# Apply the points assigner
Points['creatinine'] = points_assigner(df = LAPS2_DF, column_name = 'creatinine', splits = creatinine_splits,
 points_dict = creatinine_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# BUN_creatinine (BUN / creatinine)
BUN_creatinine_splits = [0, 25, np.inf]
BUN_creatinine_points = {0:0, 1:10}

# Apply the points assigner
Points['BUN_creatinine'] = points_assigner(df = LAPS2_DF, column_name = 'BUN_creatinine', splits = BUN_creatinine_splits,
 points_dict = BUN_creatinine_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# albumin
albumin_splits = [0, 2, 2.5, np.inf]
albumin_points = {0:31, 1:15, 2:0}

# Apply the points assigner
Points['albumin'] = points_assigner(df = LAPS2_DF, column_name = 'albumin', splits = albumin_splits,
 points_dict = albumin_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# glucose
glucose_splits = [0, 40, 60, 200, np.inf]
glucose_points = {0:10, 1:10, 2:0, 3:3}

# Apply the points assigner
Points['glucose'] = points_assigner(df = LAPS2_DF, column_name = 'glucose', splits = glucose_splits,
 points_dict = glucose_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# hematocrit
hematocrit_splits = [0, 20, 40, 50, np.inf]
hematocrit_points = {0:7, 1:8, 2:0, 3:3}

# Apply the points assigner
Points['hematocrit'] = points_assigner(df = LAPS2_DF, column_name = 'hematocrit', splits = hematocrit_splits,
 points_dict = hematocrit_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# white_blood_cells
white_blood_cells_splits = [0, 5, 13, np.inf]
white_blood_cells_points = {0:8, 1:0, 2:11}

# Apply the points assigner
Points['white_blood_cells'] = points_assigner(df = LAPS2_DF, column_name = 'white_blood_cells', splits = white_blood_cells_splits,
 points_dict = white_blood_cells_points, missingness_points = 0, high_risk_missingness_points = 32)

In [ ]:
# pCO2
pCO2_splits = [0, 35, 45, 55, 65, np.inf]
pCO2_points = {0:7, 1:0, 2:11, 3:13, 4:12}

# Apply the points assigner
Points['pCO2'] = points_assigner(df = LAPS2_DF, column_name = 'pCO2', splits = pCO2_splits,
 points_dict = pCO2_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# pO2
pO2_splits = [0, 50, 120, np.inf]
pO2_points = {0:8, 1:0, 2:12}

# Apply the points assigner
Points['pO2'] = points_assigner(df = LAPS2_DF, column_name = 'pO2', splits = pO2_splits,
 points_dict = pO2_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# Troponin_T
Troponin_T_splits = [0, 0.014, 0.070, 0.3, 1, np.inf]
Troponin_T_points = {0:0, 1:8, 2:17, 3:19, 4:25}

# Apply the points assigner
Points['Troponin_T'] = points_assigner(df = LAPS2_DF, column_name = 'Troponin_T', splits = Troponin_T_splits,
 points_dict = Troponin_T_points, missingness_points = 0, high_risk_missingness_points = 9)

In [ ]:
# temperature
temperature_splits = [0, 96, 100.5, np.inf]
temperature_points = {0:20, 1:0, 2:3}

# Apply the points assigner
Points['temperature'] = points_assigner(df = LAPS2_DF, column_name = 'temperature', splits = temperature_splits,
 points_dict = temperature_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# HR
HR_splits = [0, 60, 110, 140, np.inf]
HR_points = {0:7, 1:0, 2:7, 3:10}

# Apply the points assigner
Points['HR'] = points_assigner(df = LAPS2_DF, column_name = 'HR', splits = HR_splits,
 points_dict = HR_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# RR
RR_splits = [0, 20, 30, np.inf]
RR_points = {0:0, 1:11, 2:21}

# Apply the points assigner
Points['RR'] = points_assigner(df = LAPS2_DF, column_name = 'RR', splits = RR_splits,
 points_dict = RR_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# blood_pressure
blood_pressure_splits = [0, 75, 90, 120, 140, 160, np.inf]
blood_pressure_points = {0:22, 1:13, 2:5, 3:0, 4:8, 5:14}

# Apply the points assigner
Points['blood_pressure'] = points_assigner(df = LAPS2_DF, column_name = 'blood_pressure', splits = blood_pressure_splits,
 points_dict = blood_pressure_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# shock_index
shock_index_splits = [0, 0.65, 0.85, np.inf]
shock_index_points = {0:0, 1:8, 2:17}

# Apply the points assigner
Points['shock_index'] = points_assigner(df = LAPS2_DF, column_name = 'shock_index', splits = shock_index_splits,
 points_dict = shock_index_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# oxygen
oxygen_splits = [0, 90, 94, np.inf]
oxygen_points = {0:22, 1:12, 2:0}

# Apply the points assigner
Points['oxygen'] = points_assigner(df = LAPS2_DF, column_name = 'oxygen', splits = oxygen_splits,
 points_dict = oxygen_points, missingness_points = 0, high_risk_missingness_points = 0)

In [ ]:
# Neurological_Score
neuro_dict = {'Normal':0, "Abmiguous":16, "Abnormal":21, "Very Abnormal":36}
neuro_df = LAPS2_DF[['high_risk', 'Neurological_Score']].copy()
neuro_df['score'] = neuro_df['Neurological_Score'].map(neuro_dict)

# Assign high_risk_missingness to NA values where high_risk == 1. Fill in the points for high risk missingess
neuro_df.loc[(neuro_df['score'].isnull()) & (neuro_df['high_risk'] == 1), 'score'] = 21
neuro_df.loc[(neuro_df['score'].isnull()) & (neuro_df['high_risk'] == 0), 'score'] = 16

# Add back to the Points df
Points['neuro'] = neuro_df['score']

In [ ]:
# All relevant columns
column_list = ['Arterial_pH_Lactate', 'sodium', 'Biblirubin', 'BUN', 'creatinine',
       'BUN_creatinine', 'albumin', 'glucose', 'hematocrit',
       'white_blood_cells', 'pCO2', 'pO2', 'Troponin_T', 'temperature', 'HR',
       'RR', 'blood_pressure', 'shock_index', 'oxygen', 'neuro']

# Add a new row with the sum of each column in the list
Points['LAPS2'] = Points[column_list].apply(sum, axis=1)

In [ ]:
LAPS2 = Points[['hadm_id', 'LAPS2']]

## [7. From earlier] Predicted hospital mortality at hospital admission

In [ ]:
# assuming 'hosp' is a previously defined dictionary containing the 'admissions' dataframe
race = hosp['admissions'][['subject_id','hadm_id', 'race']].copy()

# find top 10 most popular items
top_10 = race['race'].value_counts().nlargest(10).index

# create dummies for top 10 most popular items
dummies = pd.get_dummies(race['race']).loc[:, top_10].astype(int)

# concatenate the original dataframe with the dummy dataframe
race = pd.concat([race, dummies], axis=1)

In [ ]:
# Create a neat and minimmal dataframe
Mortality_Predictor = LAPS2_DF[['subject_id', 'hadm_id', 'stay_id', 'deathin30']].copy()
Mortality_Predictor = Mortality_Predictor.merge(Points[['stay_id', 'LAPS2']], on='stay_id', how = 'left')

# Merge in the demographics
Mortality_Predictor = Mortality_Predictor.merge(derived_age, on='hadm_id', how = 'left')
Mortality_Predictor = Mortality_Predictor.merge(gender, on='subject_id', how = 'left')
Mortality_Predictor = Mortality_Predictor.merge(CCI, on='hadm_id', how = 'left')
Mortality_Predictor = Mortality_Predictor.merge(race[['hadm_id', 'WHITE', 'BLACK/AFRICAN AMERICAN',
       'OTHER', 'UNKNOWN', 'HISPANIC/LATINO - PUERTO RICAN',
       'WHITE - OTHER EUROPEAN', 'HISPANIC OR LATINO', 'ASIAN',
       'ASIAN - CHINESE', 'WHITE - RUSSIAN']], on='hadm_id', how = 'left')

# Rename the race columns
Mortality_Predictor.rename(columns = {'WHITE':'Race_1', 'BLACK/AFRICAN AMERICAN': 'Race_2', 'OTHER':'Race_3',
'UNKNOWN':'Race_4', 'HISPANIC/LATINO - PUERTO RICAN':'Race_5', 'WHITE - OTHER EUROPEAN':'Race_6', 
'HISPANIC OR LATINO':'Race_7', 'ASIAN':'Race_8', 'ASIAN - CHINESE':'Race_9', 'WHITE - RUSSIAN':'Race_10'}, inplace = True)

In [ ]:
# Age, sex, race, comorbidities, comorbidities*LAPS2, LAPS2, LAPS2^2
# Concious decision to remove [charlson_comorbidity_index + charlson_comorbidity_index*LAPS2] terms 
# These contain data based on billed diagnoses so result could be a little missleading

formula_1 = """deathin30 ~ age + gender + Race_1 + Race_2 + Race_3 + Race_4 + Race_5 + Race_6 + Race_7 + 
Race_8 + Race_9 + Race_10 + LAPS2 + LAPS2*LAPS2"""

fit = smf.logit(formula = formula_1, data = Mortality_Predictor).fit()

predictions = fit.predict()

# Binarizing the predictions
predictions_binary = (predictions > 0.5).astype(int)

# Getting actual labels
actual_labels = Mortality_Predictor['deathin30']

# calculating F1 score
f1 = f1_score(actual_labels, predictions_binary)

# Make a trim dataset
LAPS2_complete = Mortality_Predictor[['hadm_id', 'LAPS2']].copy()
LAPS2_complete['Mortality_prob'] = predictions

In [ ]:
# Reduce to smaller dataset
mortality_prob = LAPS2_complete[['hadm_id', 'Mortality_prob']]

# Putting it all together 

In [ ]:
# Base data from the icustays table
input_data = icu['icustays'][['stay_id', 'hadm_id', 'subject_id']]

# DERIVED_AGE + column for over 89
input_data = input_data.merge(derived_age, on="hadm_id", how='left')

# Merge in comorbidiy disease - CCI_export
input_data = input_data.merge(CCI, on="hadm_id", how='left')

# Merge in admission source 
input_data = input_data.merge(admission_measures, on="hadm_id", how='left')

# Merge in need for surgical or procedural intervention 
input_data = input_data.merge(surgery_out, on="hadm_id", how='left')

# Merge in severity of illness at hospital admission
input_data = input_data.merge(LAPS2, on="hadm_id", how='left')

# Merge in code status code_details.
input_data = input_data.merge(code_details, on="stay_id", how='left')
input_data['Full_code'] = input_data['Full_code'].fillna(1.0)
input_data['DNR'] = input_data['DNR'].fillna(0.0)

# Merge in predicted hospital mortality at hospital admission
input_data = input_data.merge(mortality_prob, on="hadm_id", how='left')

# Merge in severity of illness of ICU admission - sapsii
input_data = input_data.merge(sapsii, on="stay_id", how='left')

# Merge in days receiving benzodiazepines. Fill NAs with 
input_data = input_data.merge(benzos_info, on="stay_id", how='left')
input_data.benzos_days = input_data.benzos_days.fillna(0.0)

# Merge in days receiving other sedatives / non-benzodiazepines
input_data = input_data.merge(seds_info, on="stay_id", how='left')
input_data.seds_days = input_data.seds_days.fillna(0.0)

# Merge in days receiving opiates 
input_data = input_data.merge(opiates_info, on="stay_id", how='left')
input_data.opiate_days = input_data.opiate_days.fillna(0.0)

# Merge in hospital LOS. Note that I am merging on hadm_id not stay_id as hospitalisation statistic
input_data = input_data.merge(hosp_los[['hadm_id', 'hosp_LOS']], on="hadm_id", how='left')

# Merge in vital status at discharge. Note that I am merging on hadm_id not stay_id as hospitalisation statistic
input_data = input_data.merge(vital_status_discharge[['hadm_id', 'vital_status_discharge']], on="hadm_id", how='left')

# Merge in vital status 30 days after admission. Note that I am merging on hadm_id not stay_id as hospitalisation statistic
input_data = input_data.merge(deathin30_df[['hadm_id', 'deathin30']], on="hadm_id", how='left')

# Merge in discharge location. Note that I am merging on hadm_id not stay_id as hospitalisation statistic.
input_data = input_data.merge(discharge_locations[['hadm_id', 'discharged_home', 'discharged_hospice', 'discharged_skilled_facility']], on="hadm_id", how='left')

# Merge in hospital readmission within 30 days of discharge from hospital. Note that I am merging on hadm_id not stay_id as hospitalisation statistic
input_data = input_data.merge(readmission_df[['hadm_id', 'readmissionin30']], on="hadm_id", how='left')

# Check and filter out patients we don't want to include in the dataset 

In [ ]:
# Data for filtering
data_for_filtering = input_data.copy()

# Check ages
print(f"There are {len(data_for_filtering[data_for_filtering.age < 18])} patients under 18.")

# Filter out OBS 
filtered_data = data_for_filtering[~data_for_filtering.hadm_id.isin(Obstetrical_hadm_ids)].reset_index(drop = True)
print(f"There are {len(data_for_filtering) - len(filtered_data)} obstetrical patients in the dataset. Now removed.")

# Export the final dataset to your data directory

In [ ]:
# Filter and rearrange columns
Clustering_Data = filtered_data[['stay_id', 'age', 'charlson_comorbidity_index',
'ED_Admission', 'Surgery_Admission', 'LAPS2', 'Full_code', 'DNR', 'Mortality_prob', 'sapsii', 'benzos_days', 'seds_days',
'opiate_days', 'hosp_LOS', 'vital_status_discharge', 'deathin30', 'discharged_home', 'discharged_hospice',
'discharged_skilled_facility', 'readmissionin30']]

In [ ]:
# Export to csv
Clustering_Data.to_csv("data/cleaned_data.csv", index=False)